In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv("C:/Users/rajku/Documents/customer-churn-prediction/data/dataset.csv")


In [2]:
df.drop("customerID", axis=1, inplace=True)


In [3]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")


In [4]:
df["TotalCharges"].isnull().sum()


np.int64(11)

In [5]:
df["TotalCharges"] = df["TotalCharges"].fillna(df["TotalCharges"].median())


In [6]:
df.dtypes[df.dtypes == "object"]


Series([], dtype: object)

In [7]:
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})


In [8]:
categorical_cols = [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod"
]

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)


In [9]:
X = df.drop("Churn", axis=1)
y = df["Churn"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

model = LogisticRegression(max_iter=500, solver="liblinear")

print("Before training")
model.fit(X_train, y_train)
print("After training")


Before training
After training


In [10]:
from sklearn.metrics import classification_report, roc_auc_score

print(classification_report(y_test, model.predict(X_test)))
print("ROC-AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))


              precision    recall  f1-score   support

           0       0.85      0.91      0.88      1036
           1       0.70      0.57      0.63       373

    accuracy                           0.82      1409
   macro avg       0.78      0.74      0.75      1409
weighted avg       0.81      0.82      0.81      1409

ROC-AUC: 0.8617944351858562


In [11]:
y_prob = model.predict_proba(X_test)[:, 1]

# Custom threshold for business use-case
y_pred_custom = (y_prob >= 0.35).astype(int)

from sklearn.metrics import classification_report
print("Threshold = 0.35")
print(classification_report(y_test, y_pred_custom))


Threshold = 0.35
              precision    recall  f1-score   support

           0       0.89      0.81      0.85      1036
           1       0.59      0.73      0.65       373

    accuracy                           0.79      1409
   macro avg       0.74      0.77      0.75      1409
weighted avg       0.81      0.79      0.80      1409



In [ ]:
#Achieved ROC-AUC of 0.86 using Logistic Regression. Improved churn recall from 57% to 73% by tuning the decision threshold, prioritizing early churn detection over precision to align with business needs.